## Exploring The Telenor Telco Dataset

In [ ]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import os
import pickle
from datetime import datetime
from sklearn.metrics import f1_score, precision_score, recall_score
import plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode
import plotly.graph_objs as go
import seaborn as sns
import cufflinks as cf
import glob
import sys
import json
from tqdm.notebook import tqdm
cf.go_offline()
init_notebook_mode


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

### Loading the data

In [ ]:
data_path = '../../Data/Data_DK/radio_kpis_sample.csv'
#data_path = '../../Hackathon2021_Telenor/hackathon_kpis_anonymised.csv'
df = pd.read_csv(data_path, sep=";")
df_orig = df.copy()

In [ ]:
df_orig.head()

### Some info
From below we see that we have 1048575 rows of data, each with 24 columns. 

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.describe()

### Print number of nans for each column

In [ ]:
nan_values = df.isna().sum()
print(nan_values)
nan_columns = nan_values[nan_values > 0].index.tolist()
nan_columns

In [ ]:
df2 = df[df['tech'] == '2G']
nan_values = df2.isna().sum()
nan_values / df2.shape[0]

In [ ]:
df2 = df[df['tech'] == '3G']
nan_values = df2.isna().sum()
nan_values / df2.shape[0]

In [ ]:
df2 = df[df['tech'] == '4G']
nan_values = df2.isna().sum()
nan_values / df2.shape[0]

In [ ]:
# Removing data for a TEST cell 
df = df[df['cell_name'] != 'J5002D11_TEST']

# Remove 2G cells (for now)
df = df[df['tech'] != '2G']

# Some rows have columns with values -1, removing those rows
# columns with some -1 values:
# mcdr_nom_s, msdr_nom_s, msdr_nom_d, ho_nom
for c in ['mcdr_nom_s', 'msdr_nom_s', 'msdr_nom_d', 'ho_nom']:
    df = df[df[c] != -1]

# Converting a OBJECT column to FLOAT
df['thp_nom_tt_kpi'].replace('0,00E+00', 0.0, inplace=True)
df['thp_denom_tt_kpi'].replace('0,00E+00', 0.0, inplace=True)

df['thp_nom_tt_kpi'] = df['thp_nom_tt_kpi'].astype(float)
df['thp_denom_tt_kpi'] = df['thp_denom_tt_kpi'].astype(float)

# Change sector values and carrier
df['sector'] = df['sector'].apply(lambda l: int(l[-1]))
df['cell_name'] = df['cell_name'].apply(lambda l: int(l[-2]))
df = df.rename(columns={'cell_name': 'carrier'})


In [ ]:
#df['timestamp'] = df['period_start_time'].apply(lambda l: str(l[:10]) + ' ' + str(l[11:19]))
df['day_partition_key'] = df['day_partition_key'].astype(str)
df['cell_hour'] = df['cell_hour'].astype(str)
#df['cell_hour'] = df['cell_hour'].apply(lambda l: '0' + l if len(l) == 1 else l)

df['timestamp'] = df['day_partition_key'] + df['cell_hour']
df.head()

### Sorting data after timestamp

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y%m%d%H')
df = df.sort_values(by=['timestamp'])
df.tail()

### Checking the resolution 
From below we see that it is hourly, and only once is there a gap of more than an hour (2020-03-29).

In [ ]:
grouped_df = df.groupby(['timestamp'])

for timestamp, group in grouped_df:
    t = timestamp
    break

for timestamp, group in grouped_df:
    next_t = timestamp
    td = next_t - t
    td_mins = int(td.total_seconds() / 60)
    if td_mins != 60:
        print(f'Time between {t} and {timestamp} is not 1 hour, but {td_mins/60} hours.')
    t = next_t


### Checking number of cells that has data present for each timestamp

We see that the number of cells present varies a lot and flunctuates around ~100.

In [ ]:
cell_count = df_orig.groupby('cell_name').count()['gid']
cell_count[cell_count > 5000]

In [ ]:
timestamps = []
num_cells_at_time = []
for timestamp, group in grouped_df:
    timestamps.append(timestamp)
    num_cells_at_time.append(len(group))

num_cell_df = pd.DataFrame({'timestamp': timestamps, 'num_cells': num_cells_at_time})
fig = px.line(num_cell_df, x="timestamp", y='num_cells', title='Number of cells for each timestamp')
fig.show()

plt.boxplot(num_cells_at_time);
plt.title('Boxplot with number of cells for each timestamp')
plt.ylabel('# of cells')
plt.xticks([], [])
plt.show()

## Aggregating cells within same sector

### For Hackaton data
The cell_name is of the form 'XX_ija', where:

- XX in {00,01,02,..,30} denotes the site the cell belongs to;
- i in {1,2,3} denotes the sector the cell belongs to;
- j in {1,2,...} denotes the carrier;
- a in {'Z','X','Y','W','V','R','Q','P'} denotes the technology and frequency of the cell based on the table below.

### For Original data
The cell_name is of the form: '{1char}{4digits}{1char}{2chars}', where:

- 1char is Area
- 4digits is range per Area
- 1char is System (frequency/technology)
- 2chars: 1st digit is carrier layer, starting from 1. 2nd digit is sector, range 0-9

In [ ]:
df.head()

In [ ]:
is_original_data = True
df2 = df.copy()

if not is_original_data:    
    df2['site'] = df2['cell_name'].apply(lambda s: int(s.split('_')[0]))
    df2['sector'] = df2['cell_name'].apply(lambda s: int(s.split('_')[1][0]))
    df2['carrier'] = df2['cell_name'].apply(lambda s: int(s.split('_')[1][1]))
    df2['tech'] = df2['cell_name'].apply(lambda s: s.split('_')[1][2])
df2.head()

### Fill NaNs of cells 

In [ ]:
agg_df = df2.copy()
for c in nan_columns:
    if is_numeric_dtype(agg_df[c]):
        if c in ['thp_denom_tt_kpi', 'thp_nom_tt_kpi']:
            agg_func = 'mean'
        else:
            agg_func = 'median'
        agg_df[c] = agg_df[c].fillna(agg_df.groupby(['timestamp', 'site', 'sector', 'tech'])[c].transform(agg_func))
        agg_df[c] = agg_df[c].fillna(agg_df.groupby(['timestamp', 'site', 'sector'])[c].transform(agg_func))
        agg_df[c] = agg_df[c].fillna(agg_df.groupby(['timestamp', 'site'])[c].transform(agg_func))
        agg_df[c] = agg_df[c].fillna(agg_df.groupby(['site', 'sector'])[c].transform(agg_func))
        agg_df[c] = agg_df[c].fillna(agg_df.groupby(['site'])[c].transform(agg_func))
        agg_df[c] = agg_df[c].fillna(agg_df[c].median())

print(agg_df.isna().sum())
agg_df.head()

### Now aggregate across each sector

In [ ]:
agg_df.columns

In [ ]:
# To aggregate columns differently, pass this to the agg-function
#aggregate_methods = {
 #   'avail_period_duration': 'mean',
  #  'unavail_unplan_nom': 'mean',
   # 'unavail_unplan_denom': 'mean',
    #'unavail_total_nom': 'mean',
    #'unavail_total_denom': 'mean',
    #'bandwidth': 'mean'
#}

agg_df = agg_df.groupby(['timestamp', 'site', 'sector']).agg('mean').reset_index()
# Remove irrelevant columns 
agg_df.drop(columns=['gid', 'carrier'], inplace=True)
agg_df[(agg_df['site'] == 'J1824') & (agg_df['sector'] == 1)].head()

In [ ]:
grouped_agg_df = agg_df.groupby(['timestamp'])
timestamps = []
num_sectors_at_time = []
for timestamp, group in grouped_agg_df:
    timestamps.append(timestamp)
    num_sectors_at_time.append(len(group))

num_sector_df = pd.DataFrame({'timestamp': timestamps, 'num_sectors': num_sectors_at_time})
fig = px.line(num_sector_df, x="timestamp", y='num_sectors', hover_data={"timestamp": "|%B %d. %H:%M, %Y"}, title='Number of sectors for each timestamp')
fig.show()


plt.boxplot(num_sectors_at_time);
plt.title('Boxplot with number of sectors for each timestamp')
plt.ylabel('# of sectors')
plt.xticks([], [])
plt.show()

### Visualizing the number of timetamps where the sectors has data

In [ ]:
agg_df.head()

In [ ]:
plt.figure(figsize=(70, 10))
ax = sns.countplot(x="site", hue="sector", data=agg_df)
plt.legend(loc='upper right', fontsize=40)
ax.set_ylabel('# of cells', fontsize=50)
ax.xaxis.label.set_size(50)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.title('Number of timestamp where sector has data, for each site', fontsize=70)
plt.show()

In [ ]:
# Removing sites without exactly three sectors 
agg_df = agg_df[~agg_df['site'].isin(['J2964', 'J4608', 'J5004', 'J4969', 'J8062'])]

In [ ]:
plt.figure(figsize=(70, 10))
ax = sns.countplot(x="site", hue="sector", data=agg_df)
plt.legend(loc='upper right', fontsize=40)
ax.set_ylabel('# of cells', fontsize=50)
ax.xaxis.label.set_size(50)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.title('Number of timestamp where sector has data, for each site', fontsize=70)
plt.show()

### Visualizing data for one of the sectors

In [ ]:
def visualize_sector(df, site, sector, column):
    if sector is not None:
        sector_df = df[(df['site'] == site) & (df['sector'] == sector)]
    else:
        sector_df = df[(df['site'] == site)]
    fig = px.line(sector_df, x="timestamp", y=column)
    fig.show()

In [ ]:
agg_df.columns

In [ ]:
visualize_sector(agg_df, 'J0847', 1, 'avail_period_duration')

## Creating dataset 
In **agg_df**, each row corresponds to aggregated data from all cells within a sector, at a specific timestamp. However, not all sectors do have at least one active cell at all timestamps. If a sector contains no active cells for a timestamp, then a row for that sector at that timestamp does not exist. To make the resolution (every hour) and shape (there is a row for all sectors for all timestamps) consistent, we must insert rows. 

In [ ]:
agg_df.columns

In [ ]:
default_values = {
    'avail_period_duration': 60, # maybe exclude
    'unavail_unplan_nom': 0,  # maybe exclude
    'unavail_unplan_denom': 60, # maybe exclude
    'unavail_total_nom': 0, # maybe maybe exclude
    'unavail_total_denom': 60,# maybe maybe exclude
    'bandwidth': 0, # change to max of sector # maybe exclude
    'mcdr_denom': 0, 
    'mcdr_nom_s': 0,
    'mcdr_nom_d': 0,
    'msdr_denom': 0, 
    'msdr_nom_s': 0,
    'msdr_nom_d': 0,
    'thp_denom_tt_kpi': 0, # change to max of sector
    'thp_nom_tt_kpi': 0,
    'ho_denom': 0,
    'ho_nom': 0
}

# change thp, ho, msdr, mcdr to rates 

sector_bandwidth_max = agg_df.groupby(['site', 'sector']).agg('max')['bandwidth'].to_dict()
sector_thp_denom_max = agg_df.groupby(['site', 'sector']).agg('max')['thp_denom_tt_kpi'].to_dict()
# default_values_list = [default_values[c] for c in agg_df.columns[3:]]

In [ ]:
is_present_set = set([tuple(x) for x in agg_df[['timestamp', 'site', 'sector']].values])

In [ ]:
agg_df.columns

In [ ]:
datetime_start = agg_df.head(1)['timestamp'].iloc[0]
datetime_end = agg_df.tail(1)['timestamp'].iloc[0]

all_timestamps = pd.date_range(datetime_start, datetime_end, freq="H")
all_sites = agg_df['site'].unique()
all_sectors = agg_df['sector'].unique()

rows_to_add = []
for timestamp in tqdm(all_timestamps):
    for site in all_sites:
        for sector in all_sectors:
            if (timestamp, site, sector) not in is_present_set:
                bandwidth_default = sector_bandwidth_max[(site, sector)]
                thp_default = sector_thp_denom_max[(site, sector)]
                default_values['bandwidth'] = bandwidth_default
                default_values['thp_denom_tt_kpi'] = thp_default
                default_values_list = [default_values[c] for c in agg_df.columns[3:]]
                row_insert = [timestamp, site, sector]
                row_insert.extend(default_values_list)
                rows_to_add.append(row_insert)

In [ ]:
rows_to_add_df = pd.DataFrame(rows_to_add, columns=agg_df.columns)

In [ ]:
agg_df_filled = agg_df.append(rows_to_add_df)
agg_df_filled = agg_df_filled.sort_values(by=['timestamp', 'site', 'sector'])

In [ ]:
print(agg_df_filled.shape)
agg_df_filled.head(6)

#### Check that new dataframe now has constant number of sectors for each timestamp

In [ ]:
grouped_agg_df = agg_df_filled.groupby(['timestamp'])
timestamps = []
num_sectors_at_time = []
for timestamp, group in grouped_agg_df:
    timestamps.append(timestamp)
    num_sectors_at_time.append(len(group))

num_sector_df = pd.DataFrame({'timestamp': timestamps, 'num_sectors': num_sectors_at_time})
fig = px.line(num_sector_df, x="timestamp", y='num_sectors', hover_data={"timestamp": "|%B %d. %H:%M, %Y"}, title='Number of sectors for each timestamp')
fig.show()

#### Visualize column again

In [ ]:
agg_df_filled.columns

In [ ]:
# Cols to keep:
cols_to_keep = ['timestamp', 'site', 'sector', 'avail_period_duration', 'unavail_unplan_nom', 'unavail_unplan_denom', 'mcdr_denom', 'msdr_denom', 'msdr_nom_s', 'msdr_nom_d', 'ho_denom', 'ho_nom']

In [ ]:
visualize_sector(agg_df, 'J0847', 1, 'avail_period_duration')

# Group Sectors Into the Final Dataset
## Alternative 1: 
Group the data to get a series of graph signals with shape: (# of timestamps, # of sectors, # of features), see figure below: <img src="https://i.imgur.com/1izToWi.png" width=750 height=750 />.

In [ ]:
def save_metadata(df, path):
    metadata = {
    'start': str(df.head(1)['timestamp'].iloc[0]),
    'end': str(df.tail(1)['timestamp'].iloc[0]),
    'resolution_minutes': (df.iloc[1, 0] - df.iloc[0, 0]).seconds / 60,
    'columns': df.columns.tolist()}
    
    with open(path, 'w') as file:
        json.dump(metadata, file, indent=2)

In [ ]:
grouped_agg_df = agg_df_filled.groupby(['timestamp'])
graph_signals = []

for timestamp, group in tqdm(grouped_agg_df):
    graph_signals.append(group)

graph_signals = np.array(graph_signals)

In [ ]:
print(metadata)
print(graph_signals.shape)
save = False
save_path = '../datasets/telenor/'
os.makedirs(save_path, exist_ok=True)

if save:
    save_metadata(agg_df_filled, f'{save_path}/graph_signals_metadata.txt') 
    np.save(f'{save_path}/graph_signals.npy', graph_signals)

## Alternative 2: 
'Augment' each sector with the sectors of the same site. 

In [ ]:
print(agg_df_filled.shape)
agg_df_filled.columns

In [ ]:
# Reduce number of cols
agg_df_filled = agg_df_filled[cols_to_keep]
agg_df_filled.columns

In [ ]:
site_grouped = agg_df_filled.groupby(['site', 'timestamp'])
sectors_df = []

for (site, timestamp), group in tqdm(site_grouped):
    # Merge the three sectors
    group = group.drop(columns=['sector'])
    s1, s2, s3 = group.iloc[0, :], group.iloc[1, :], group.iloc[2, :]

    sectors_comb = {}
    sectors_comb['timestamp'] =  str(timestamp)
    sectors_comb['site'] = site

    for i, s in enumerate([s1, s2, s3], start=1):
        for c in group.columns[2:]:
            sectors_comb[f'{c}_{i}'] = s[c]
    
    sectors_df.append(sectors_comb)

sectors_df = pd.DataFrame(sectors_df, index=range(len(sectors_df)))
sectors_df['timestamp'] = pd.to_datetime(sectors_df['timestamp'])
sectors_df = sectors_df.sort_values(by=['site', 'timestamp'])
display(sectors_df.head())

In [ ]:
visualize_sector(sectors_df, 'J0847', None, 'mcdr_denom_3')

In [ ]:
print(sectors_df.shape)
save = True
save_path = '../datasets/telenor/site_data/'
os.makedirs(save_path, exist_ok=True)
if save:
    save_metadata(sectors_df, f'{save_path}/metadata.txt') 
    grouped_sectors_df = sectors_df.groupby('site')
    for site, group in grouped_sectors_df:
        np.save(f'{save_path}/{site}.npy', group.values)